### Install

In [ ]:
%pip install -U langchain-community langgraph langchain-google-genai tavily-python langgraph-checkpoint-sqlite

In [ ]:
%pip install python-dotenv

In [2]:
import getpass
import os
from dotenv import load_dotenv

os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

### Create model

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

### Project Info Agent
Extracts problem and make/model

In [ ]:
messages = [
    (
        "system",
        "You are a 'Project-info' agent that extracts a *problem* and *make/model* from a user's query. Return Unknown if inappropriate query."

    ),
    ("human", "How do I clean my MSI GS66 laptop's fan?"),

]

response = llm.invoke(messages)
response
response.content

### Project Info Agent


In [38]:
from langchain_core.prompts import ChatPromptTemplate

project_info_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a 'Project-info' agent that extracts a *problem* and" 
            "*make/model* from a user's query. Return Unknown if inappropriate"
            "query. Return in valid JSON format"
            
        ),
        ("human", "{input}"),
    ]

)


### Instructions Agent


In [59]:
instruction_agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an 'Instruction-generator' agent that "
            "provides a list of step-by-step instructions to solve the"
            " given *problem* for the specified *make/model*. Provide it in JSON format with step1, step2, ... as the keys"
        ),
        ("human", "Problem: {problem}, Make/Model: {make_model}"),
    ]
)

### Chaining

In [57]:
# user_input = "How do I clean my MSI GS66 laptop's fan?"
# user_input = "What is the weather today"
user_input = "Where can I find the memory card location on my nintendo switch?"

project_info_chain = project_info_prompt | llm
project_info_response = project_info_chain.invoke({"input": user_input})

project_info_content = project_info_response.content

project_info_content = project_info_content.replace("```", "")
project_info_content = project_info_content.replace("json", "")
project_info = eval(project_info_content)
print(project_info_content)
print(project_info)


{
  "problem": "memory card location",
  "make/model": "nintendo switch"
}

{'problem': 'memory card location', 'make/model': 'nintendo switch'}


In [ ]:
problem = project_info.get("problem", "Unknown")
make_model = project_info.get("make/model", "Unknown")

if problem != "Unknown" and make_model != "Unknown":
    instruction_chain = instruction_agent_prompt | llm
    instruction_response = instruction_chain.invoke({
        "problem": problem,
        "make_model": make_model
    })
    instructions = instruction_response.content.replace("json", "")
    instructions = instructions.replace("```", "")
    print(instructions)

# print(instruction_response)

```json
{
  "step1": "Power off your Nintendo Switch completely. This is crucial to avoid data corruption.",
  "step2": "Locate the game card slot. It's usually on the top edge of the console, on the right side.",
  "step3": "Look for a small flap or cover near the game card slot. This is the microSD card slot cover.",
  "step4": "Gently open the microSD card slot cover. You might need to use your fingernail or a small, non-metallic tool.",
  "step5": "The microSD card slot is located underneath the flap. You will see the edge of the microSD card if one is inserted.",
  "step6": "To remove the microSD card, gently push it in slightly until you hear or feel a click. This will release the card.",
  "step7": "Carefully pull the microSD card out of the slot."
}
```
